In [3]:
#%% Imports
from pathlib import Path
import numpy as np

# Import "local" packages
import hyspec_io, hyspec_ml

In [4]:
# Define / create paths
image_base_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/Hyperspectral/20230620/2a_Rrs')

image_output_dir = image_base_dir.parent / '3a_Rrs_SGC'
pca_output_dir = image_output_dir / 'PCA_SGC_Models'
image_output_dir.mkdir(exist_ok=True)
pca_output_dir.mkdir(exist_ok=True)

In [5]:
# Define image groups (images taken during same flight are in same group)
filename_bases = ('manual_Pika_L_',) + ('Skardskjaringen_Pika_L_',)*4 + ('SkardskjaringenSor_Pika_L_',)*2
file_number_ranges = ((25,29),(2,25),(26,54),(55,82),(83,98),(1,23),(24,41))
filename_endings = ('-Crop Wavelengths.bip.hdr',)*7
sgc_training_file_numbers = ((25,28),(6,10,19),(32,41,48),(55,62,68,75,81),(85,88,93),(2,8,18,19,23),(29,34,38))

In [10]:
# Loop over all image groups, and images within each group
for filename_base, file_number_range, filename_end, train_file_nums in zip(
    filename_bases, file_number_ranges, filename_endings, sgc_training_file_numbers):

    # Define file names
    image_file_names = [filename_base + str(i) + filename_end for i in range(file_number_range[0],file_number_range[1]+1)]
    image_file_paths = [image_base_dir / im_file for im_file in image_file_names]
    training_files = [filename_base + str(i) + filename_end for i in train_file_nums]
    training_file_paths = [image_base_dir / im_file for im_file in training_files]
    
    # Validate that training images are among images in group
    if not all([tfp in image_file_paths for tfp in training_file_paths]):
        print('WARNING: Training file paths not among image files in group! Continuing to next group...')
        continue

    # Validate that images exist
    if not all([ifp.exists() for ifp in image_file_paths]):
        print('WARNING: Image file paths not valid! Continuing to next group...')
        continue

    # Train PCA model
    W_pca, X_mean = hyspec_ml.train_image_pca(image_paths=training_file_paths,random_sample_frac=0.1,
                                              verbose=True,normalize=False)

    # Save PCA model
    pca_file_name = f'{filename_base}PCA-SGC_{file_number_range[0]:02d}-{file_number_range[1]:02d}.npz'
    pca_file_path = pca_output_dir / pca_file_name
    np.savez(pca_file_path,
        W_pca = W_pca,
        X_mean = X_mean)
    
    # Apply PCA-SGC to all images and save corrected images:
    for i,image_file_path in enumerate(image_file_paths):
        print(f'Processing file {image_file_path.name}, number {i+1} of {len(image_file_paths)} in group')
        (image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(image_file_path)
        image_noglint = hyspec_ml.remove_sun_glint_pca(image,W_pca,X_mean)

        sgc_file_base, sgc_file_ext = image_file_path.name.split('.',maxsplit=1)
        hyspec_ml.save_pca_scg_image_envi(image_noglint,metadata,image_output_dir,
                                          sgc_file_base,header_file_ext=sgc_file_ext)